In [4]:
from typing import Callable, Mapping, TypeVar, Sequence, Any, Iterable
from dataclasses import dataclass
from haskellian import either as E
from pipeteer import Pipeline, SimpleQueue, Workflow, Wrapped, QueueKV, make_queues, flatten_queues
from example_workflow import Artifacts, Input
from prettyprinter import pformat

In [5]:
Qin = QueueKV.sqlite(Input, 'queue.sqlite', 'Qin')

In [7]:
for i in range(1, 10):
  await Qin.push(f'task{i}', Input(f'img{i}'.encode()))